In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.optimizers import Adam
import tensorflow as tf
from keras_tuner import RandomSearch
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import random

# Define the file paths for each period
file_paths = {
    'bear_market_1': '../../../Data/Final_df/bear_market_1_classification.csv',
    'bear_market_2': '../../../Data/Final_df/bear_market_2_classification.csv',
    'bull_market_1': '../../../Data/Final_df/bull_market_1_classification.csv',
    'bull_market_2': '../../../Data/Final_df/bull_market_2_classification.csv'
}

# Initialize the results dictionary
results = {}

# Seed value for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Normalize the data
columns_to_scale = ['Open', 'High', 'Low', 'Close', 'Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users']

scaler = MinMaxScaler(feature_range=(0, 1))

# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# Forecast Horizon
window_size = 3
close_index = 22

def build_model(hp):
    model = Sequential()
    filters = hp.Int('filters', min_value=10, max_value=150, step=5)
    pool = hp.Choice('pool_size', values=[2, 3])
    model.add(Conv2D(filters=filters,
                    kernel_size=3, activation='relu', padding='same', input_shape=(window_size, len(columns_to_scale), 1)))
    model.add(MaxPooling2D(pool_size=pool, padding='same'))
    
    model.add(Conv2D(filters=filters, 
                     kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=pool, padding='same'))

    model.add(Conv2D(filters=filters, 
                     kernel_size=3, activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=pool,padding='same'))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Function to fit the model and evaluate metrics
def fit_and_evaluate(data, scaler, window_size, close_index, project_name):
    data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])
    training_size = int(len(data) * 0.9)
    training_data = data[:training_size]
    test_data = data[training_size:]
    train_data = training_data[:int(len(training_data) * 0.9)]
    val_data = training_data[int(len(training_data) * 0.9):]

    X_train, y_train = create_dataset(train_data, window_size, close_index)
    X_test, y_test = create_dataset(test_data, window_size, close_index)
    X_val, y_val = create_dataset(val_data, window_size, close_index)
    X_train_full, y_train_full = create_dataset(training_data, window_size, close_index)

    X_train = X_train.reshape((X_train.shape[0], window_size, X_train.shape[2], 1))
    X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))
    X_val = X_val.reshape((X_val.shape[0], window_size, X_val.shape[2], 1))
    X_train_full = X_train_full.reshape((X_train_full.shape[0], window_size, X_train_full.shape[2], 1))

    tuner = RandomSearch(
        build_model,
        objective='val_accuracy',
        max_trials=200,
        executions_per_trial=1,
        directory='my_dir',
        project_name=project_name,
        overwrite=False
    )

    early_stopping = EarlyStopping(monitor='val_accuracy', patience=15, restore_best_weights=True)

    tuner.search(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), verbose=2, callbacks=[early_stopping])


    accuracy_list, f1_list, precision_list, recall_list = [], [], [], []
    for _ in range(10):
        best_hps = tuner.get_best_hyperparameters()[0]
        best_model = tuner.hypermodel.build(best_hps)
        history = best_model.fit(X_train_full, y_train_full, epochs=250, batch_size=32, verbose=2)
        predictions = best_model.predict(X_test)
        test_pred = (predictions > 0.5).astype(int)

        accuracy_list.append(accuracy_score(y_test, test_pred))
        f1_list.append(f1_score(y_test, test_pred))
        precision_list.append(precision_score(y_test, test_pred))
        recall_list.append(recall_score(y_test, test_pred))

    return {
        'accuracy': np.mean(accuracy_list),
        'f1_score': np.mean(f1_list),
        'precision': np.mean(precision_list),
        'recall': np.mean(recall_list)
    }

# Process each period and store results
for period, file_path in file_paths.items():
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    project_name = f'CNN-3D-CLASS-{period}'
    results[period] = fit_and_evaluate(data, scaler, window_size, close_index, project_name)

# Create a DataFrame from the results
results_df = pd.DataFrame(results).T
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('CNN_BL_BR_performance_summary.csv')


Reloading Tuner from my_dir/CNN-3D-CLASS-bear_market_1/tuner0.json
Epoch 1/250


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 - 0s - 44ms/step - accuracy: 0.4756 - loss: 0.7135
Epoch 2/250
11/11 - 0s - 4ms/step - accuracy: 0.5122 - loss: 0.6927
Epoch 3/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6928
Epoch 4/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6932
Epoch 5/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6934
Epoch 6/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 7/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 8/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 9/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 10/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 11/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 12/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 13/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 14/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 15/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 - 1s - 50ms/step - accuracy: 0.4634 - loss: 0.7141
Epoch 2/250
11/11 - 0s - 3ms/step - accuracy: 0.5061 - loss: 0.6929
Epoch 3/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6927
Epoch 4/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6928
Epoch 5/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6930
Epoch 6/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6932
Epoch 7/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6934
Epoch 8/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6935
Epoch 9/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6935
Epoch 10/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6935
Epoch 11/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 12/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 13/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 14/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 - loss: 0.6936
Epoch 15/250
11/11 - 0s - 3ms/step - accuracy: 0.5122 

...

31/31 - 0s - 14ms/step - accuracy: 0.5254 - loss: 0.6928
Epoch 2/250
31/31 - 0s - 3ms/step - accuracy: 0.5306 - loss: 0.6912
Epoch 3/250
31/31 - 0s - 2ms/step - accuracy: 0.5296 - loss: 0.6903
Epoch 4/250
31/31 - 0s - 1ms/step - accuracy: 0.5348 - loss: 0.6895
Epoch 5/250
31/31 - 0s - 1ms/step - accuracy: 0.5421 - loss: 0.6886
Epoch 6/250
31/31 - 0s - 1ms/step - accuracy: 0.5389 - loss: 0.6866
Epoch 7/250
31/31 - 0s - 2ms/step - accuracy: 0.5452 - loss: 0.6854
Epoch 8/250
31/31 - 0s - 2ms/step - accuracy: 0.5576 - loss: 0.6835
Epoch 9/250
31/31 - 0s - 1ms/step - accuracy: 0.5639 - loss: 0.6824
Epoch 10/250
31/31 - 0s - 1ms/step - accuracy: 0.5649 - loss: 0.6808
Epoch 11/250
31/31 - 0s - 1ms/step - accuracy: 0.5670 - loss: 0.6796
Epoch 12/250
31/31 - 0s - 2ms/step - accuracy: 0.5794 - loss: 0.6778
Epoch 13/250
31/31 - 0s - 1ms/step - accuracy: 0.5846 - loss: 0.6772
Epoch 14/250
31/31 - 0s - 1ms/step - accuracy: 0.5857 - loss: 0.6761
Epoch 15/250
31/31 - 0s - 2ms/step - accuracy: 0.5877 

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 - 0s - 14ms/step - accuracy: 0.5275 - loss: 0.6924
Epoch 2/250
31/31 - 0s - 1ms/step - accuracy: 0.5306 - loss: 0.6907
Epoch 3/250
31/31 - 0s - 3ms/step - accuracy: 0.5306 - loss: 0.6904
Epoch 4/250
31/31 - 0s - 2ms/step - accuracy: 0.5306 - loss: 0.6897
Epoch 5/250
31/31 - 0s - 2ms/step - accuracy: 0.5306 - loss: 0.6890
Epoch 6/250
31/31 - 0s - 1ms/step - accuracy: 0.5348 - loss: 0.6882
Epoch 7/250
31/31 - 0s - 1ms/step - accuracy: 0.5369 - loss: 0.6871
Epoch 8/250
31/31 - 0s - 1ms/step - accuracy: 0.5306 - loss: 0.6859
Epoch 9/250
31/31 - 0s - 1ms/step - accuracy: 0.5400 - loss: 0.6847
Epoch 10/250
31/31 - 0s - 1ms/step - accuracy: 0.5618 - loss: 0.6830
Epoch 11/250
31/31 - 0s - 1ms/step - accuracy: 0.5701 - loss: 0.6815
Epoch 12/250
31/31 - 0s - 1ms/step - accuracy: 0.5763 - loss: 0.6796
Epoch 13/250
31/31 - 0s - 1ms/step - accuracy: 0.5742 - loss: 0.6779
Epoch 14/250
31/31 - 0s - 2ms/step - accuracy: 0.5742 - loss: 0.6766
Epoch 15/250
31/31 - 0s - 1ms/step - accuracy: 0.5805 

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


31/31 - 1s - 23ms/step - accuracy: 0.5317 - loss: 0.6925
Epoch 2/250
31/31 - 0s - 2ms/step - accuracy: 0.5306 - loss: 0.6913
Epoch 3/250
31/31 - 0s - 1ms/step - accuracy: 0.5306 - loss: 0.6902
Epoch 4/250
31/31 - 0s - 2ms/step - accuracy: 0.5306 - loss: 0.6896
Epoch 5/250
31/31 - 0s - 2ms/step - accuracy: 0.5317 - loss: 0.6889
Epoch 6/250
31/31 - 0s - 1ms/step - accuracy: 0.5306 - loss: 0.6881
Epoch 7/250
31/31 - 0s - 2ms/step - accuracy: 0.5389 - loss: 0.6874
Epoch 8/250
31/31 - 0s - 2ms/step - accuracy: 0.5389 - loss: 0.6862
Epoch 9/250
31/31 - 0s - 2ms/step - accuracy: 0.5545 - loss: 0.6851
Epoch 10/250
31/31 - 0s - 1ms/step - accuracy: 0.5556 - loss: 0.6837
Epoch 11/250
31/31 - 0s - 1ms/step - accuracy: 0.5701 - loss: 0.6822
Epoch 12/250
31/31 - 0s - 1ms/step - accuracy: 0.5732 - loss: 0.6809
Epoch 13/250
31/31 - 0s - 1ms/step - accuracy: 0.5805 - loss: 0.6795
Epoch 14/250
31/31 - 0s - 1ms/step - accuracy: 0.5836 - loss: 0.6780
Epoch 15/250
31/31 - 0s - 1ms/step - accuracy: 0.5857 

In [3]:
# Save the besto model for each period
# Bear Market 1
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNN-3D-CLASS-bear_market_1',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_CNN-3D-CLASS-bear_market_1.keras')

# Bear Market 2
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNN-3D-CLASS-bear_market_2',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_CNN-3D-CLASS-bear_market_2.keras')

# Bull Market 1
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNN-3D-CLASS-bull_market_1',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_CNN-3D-CLASS-bull_market_1.keras')

# Bull Market 2
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNN-3D-CLASS-bull_market_2',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_CNN-3D-CLASS-bull_market_2.keras')

Reloading Tuner from my_dir/CNN-3D-CLASS-bear_market_1/tuner0.json
Reloading Tuner from my_dir/CNN-3D-CLASS-bear_market_2/tuner0.json
Reloading Tuner from my_dir/CNN-3D-CLASS-bull_market_1/tuner0.json


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Reloading Tuner from my_dir/CNN-3D-CLASS-bull_market_2/tuner0.json
